In [ ]:
# Author: Hamaad Musharaf Shah.
# The following references were used.
# https://keras.io
# https://blog.keras.io/building-autoencoders-in-keras.html
# https://stackoverflow.com/questions/42177658/how-to-switch-backend-with-keras-from-tensorflow-to-theano
# https://towardsdatascience.com/learning-rate-schedules-and-adaptive-learning-rate-methods-for-deep-learning-2c8f433990d1
# http://scikit-learn.org/stable/
# Book: Ian Goodfellow, Yoshua Bengio and Aaron Courville, "Deep Learning" - http://www.deeplearningbook.org
# Book: Aurelien Geron, "Hands-On Machine Learning with Scikit-Learn & Tensorflow" - https://www.amazon.co.uk/Hands-Machine-Learning-Scikit-Learn-TensorFlow/dp/1491962291

import os
import math
import sys
import importlib

import numpy as np

from sklearn import linear_model
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
from sklearn.metrics import roc_auc_score

from keras import backend as bkend

from autoencoders_keras.vanilla_autoencoder import VanillaAutoencoder
from autoencoders_keras.convolutional_autoencoder import ConvolutionalAutoencoder
from autoencoders_keras.seq2seq_autoencoder import Seq2SeqAutoencoder
from autoencoders_keras.variational_autoencoder import VariationalAutoencoder

os.environ["KERAS_BACKEND"] = "tensorflow"
importlib.reload(bkend)

mnist = fetch_mldata("MNIST original")
X, y = mnist["data"], mnist["target"]
X = X.astype("float32")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=666)
min_X_train = np.apply_along_axis(func1d=np.min, axis=0, arr=X_train)
max_X_train = np.apply_along_axis(func1d=np.max, axis=0, arr=X_train) 
range_X_train = max_X_train - min_X_train + sys.float_info.epsilon
X_train = (X_train - min_X_train) / range_X_train
X_test = (X_test - min_X_train) / range_X_train

scaler_autoencoder = MinMaxScaler(feature_range=(0.0, 1.0))
scaler_classifier = MinMaxScaler(feature_range=(0.0, 1.0))
logistic = linear_model.LogisticRegression(random_state=666)

lb = LabelBinarizer()
lb = lb.fit(y_train.reshape(y_train.shape[0], 1))

In [ ]:
pipe_base = Pipeline(steps=[("scaler_classifier", scaler_classifier),
                            ("classifier", logistic)])
pipe_base = pipe_base.fit(X_train, y_train)

metric_base = roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                            pipe_base.predict_proba(X_test), 
                            average="weighted")
print(metric_base)

In [ ]:
autoencoder = VanillaAutoencoder(n_feat=X_train.shape[1],
                                 n_epoch=50,
                                 batch_size=100,
                                 encoder_layers=3,
                                 decoder_layers=3,
                                 n_hidden_units=int(X_train.shape[1] / 2),
                                 encoding_dim=int(X_train.shape[1] / 2))

pipe_autoencoder = Pipeline(steps=[("scaler_autoencoder", scaler_autoencoder),
                                   ("autoencoder", autoencoder),
                                   ("scaler_classifier", scaler_classifier),
                                   ("classifier", logistic)])

pipe_autoencoder = pipe_autoencoder.fit(X_train, y_train)

metric_autoencoder = roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                                   pipe_autoencoder.predict_proba(X_test), 
                                   average="weighted")
print(metric_autoencoder)

In [ ]:
noise = 0.01 * np.reshape(np.random.uniform(low=0.0, 
                                            high=1.0, 
                                            size=X_train.shape[0] * X_train.shape[1]), 
                          [X_train.shape[0], X_train.shape[1]])

denoising_autoencoder = VanillaAutoencoder(n_feat=X_train.shape[1],
                                           n_epoch=50,
                                           batch_size=100,
                                           encoder_layers=3,
                                           decoder_layers=3,
                                           n_hidden_units=int(X_train.shape[1] / 2),
                                           encoding_dim=int(X_train.shape[1] / 2))

pipe_denoising_autoencoder = Pipeline(steps=[("scaler_autoencoder", scaler_autoencoder),
                                             ("autoencoder", denoising_autoencoder),
                                             ("scaler_classifier", scaler_classifier),
                                             ("classifier", logistic)])

pipe_denoising_autoencoder = pipe_denoising_autoencoder.fit(X_train + noise, y_train)

metric_denoising_autoencoder = roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                                             pipe_denoising_autoencoder.predict_proba(X_test), 
                                             average="weighted")
print(metric_denoising_autoencoder)

In [ ]:
convolutional_autoencoder = ConvolutionalAutoencoder(input_shape=(int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))),
                                                     n_epoch=50,
                                                     batch_size=100,
                                                     encoder_layers=3,
                                                     decoder_layers=3,
                                                     filters=100,
                                                     kernel_size=5,
                                                     strides=1,
                                                     pool_size=4,
                                                     encoding_dim=100)

pipe_convolutional_autoencoder = Pipeline(steps=[("autoencoder", convolutional_autoencoder),
                                                 ("scaler_classifier", scaler_classifier),
                                                 ("classifier", logistic)])

pipe_convolutional_autoencoder = pipe_convolutional_autoencoder.fit(np.reshape(X_train, [X_train.shape[0], int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))]), 
                                                                    y_train)

metric_convolutional_autoencoder = roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                                                 pipe_convolutional_autoencoder.predict_proba(np.reshape(X_test, [X_test.shape[0], int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))])), 
                                                 average="weighted")
print(metric_convolutional_autoencoder)

In [ ]:
seq2seq_autoencoder = Seq2SeqAutoencoder(input_shape=(int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))),
                                         n_epoch=2,
                                         batch_size=100,
                                         encoder_layers=3,
                                         decoder_layers=3,
                                         n_hidden_units=100,
                                         encoding_dim=100)

pipe_seq2seq_autoencoder = Pipeline(steps=[("autoencoder", seq2seq_autoencoder),
                                           ("scaler_classifier", scaler_classifier),
                                           ("classifier", logistic)])

pipe_seq2seq_autoencoder = pipe_seq2seq_autoencoder.fit(np.reshape(X_train, [X_train.shape[0], int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))]),
                                                        y_train)

metric_seq2seq_autoencoder = roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                                           pipe_seq2seq_autoencoder.predict_proba(np.reshape(X_test, [X_test.shape[0], int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))])), 
                                           average="weighted")
print(metric_seq2seq_autoencoder)

In [ ]:
variational_autoencoder = VariationalAutoencoder(n_feat=X_train.shape[1],
                                                 n_epoch=50,
                                                 batch_size=100,
                                                 encoder_layers=3,
                                                 decoder_layers=3,
                                                 n_hidden_units=int(X_train.shape[1] / 2),
                                                 encoding_dim=int(X_train.shape[1] / 2))

pipe_variational_autoencoder = Pipeline(steps=[("scaler_autoencoder", scaler_autoencoder),
                                               ("autoencoder", variational_autoencoder),
                                               ("scaler_classifier", scaler_classifier),
                                               ("classifier", logistic)])

pipe_variational_autoencoder = pipe_variational_autoencoder.fit(X_train, y_train)

metric_variational_autoencoder = roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                                               pipe_variational_autoencoder.predict_proba(X_test), 
                                               average="weighted")
print(metric_variational_autoencoder)

In [ ]:
# In development.
# Some transfer learning stuff.
# Probably will come in as another repo.
from keras.datasets import cifar10

cifar = cifar10.load_data()
(x_train, y_train), (x_test, y_test) = cifar
cifar[0][0][0, :, :, 0]
x_train.shape[1:]

autoencoder.layers[0:10]

for layer in autoencoder.layers[0:10]:
    layer.trainable = False

autoencoder.layers[9].output

classifier_output = autoencoder.layers[9].output
classifier_output = Dense(100, activation="elu")(classifier_output)
classifier_output = Dense(100, activation="elu")(classifier_output)
classifier_output = Dense(10, activation="softmax")(classifier_output)

model_final = Model(autoencoder.input, classifier_output)
model_final.compile(loss = "categorical_crossentropy", 
                    optimizer=keras.optimizers.Adam(), 
                    metrics=["accuracy"])

model_final.fit(X_train, keras.utils.to_categorical(y_train, 10),
                validation_split=0.3,
                epochs=50,
                batch_size=100,
                shuffle=True,
                callbacks=callbacks_list, 
                verbose=2)

model_final.predict(X_test)

from sklearn.metrics import roc_auc_score
roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
              model_final.predict(X_test), 
              average="weighted")